In [1]:
import sys
import os
from mywikicorpus import MyWikiCorpus
import pickle
import re
import timeit
import codecs
import struct
import math
import collections
import pymorphy2
from IPython.display import display, HTML
morph = pymorphy2.MorphAnalyzer()

In [3]:
def make_wiki_corpus(in_file:str,out_file_pattern:str)->(list,list):
    """
    Парсит указанный архив(дамп), собирает общую статистику и записывает все в файл
    --------------
    Params
    -------
        in_file:str
            Путь к файлу(архиву),содержимое которого необходимо распарсить
        out_file: str
            Путь в файлу для записи распарсенного содержимого
    Returns
    -------
        all_words:list
            Список слов, содержащийся во всех документах
        statistic_dict:dict
            Словарь со статистикой по документам
            
    Exceptions
    ----------
        FileNotFoundError
    """
    if not os.path.exists(in_file):
        raise FileNotFoundError("Файла в указанной директории нет")
        
    wiki = MyWikiCorpus(in_file,lemmatize = False,dictionary = {})
    texts = list()
    documents_len = list()
    index = 0
    doc_names = {}
    for article in wiki.get_texts():
        #stringlist=[x.decode('utf-8') for x in article[0]]
        #out_file = "{0}{1}.txt".format(out_file_pattern,index)
        #with open(out_file,"w",encoding = "utf-8") as stream:
         #   stream.write(" ".join(stringlist))
        #print(article[1])
        doc_names[index] = article[1][1]
        print(article[1][1])
        #texts.append(" ".join(stringlist)) 
        index+=1
        if index == 39999:
            break
        #documents_len.append(len(article))
    with open('headers.pickle','wb') as stream:
        pickle.dump(doc_names,stream)
    #return texts,documents_len

In [ ]:
words,counts = make_wiki_corpus("data/ruwiki-20181120-pages-articles-multistream.xml.bz2","data/text/wiki")

In [5]:
param = {}
param['IGNORE_STOPWORDS'] = False
param['IGNORE_SINGLE'] = False
param['BYTE_OFFSET'] = 2
param['BYTE_SIZE'] = 2

In [85]:
class BooleanIndexer(object):
    def __init__(self,document_directory,dictionary_file,postings_file,param_dict = param):
        self.param_dict = param_dict
        self.document_directory = document_directory
        self.dictionary_file = dictionary_file
        self.postings_file = postings_file
    
    def boolean_index(self):
        docID_list = [idx for idx in range(40000)]
        dictionary = {}
        dictionary_dic = {}
        for docID in docID_list:
            print(docID)
            file_path = self.document_directory + str(docID)
            file_path+='.txt'
            print(file_path)
            if(os.path.isfile(file_path)):
                file = codecs.open(file_path,encoding = 'utf-8')
                document = file.read()
                sentences = document.split('.')
                sentences = list(filter(lambda x: x != "",sentences))
                for idx in range(len(sentences)):
                    tokens = sentences[idx].split(' ')
                #one step indexing
                    t = []
                    for token in tokens:
                        t.extend(token.split('-'))
                    tokens = t
                    for word in tokens:
                        word = re.sub('\W+','',word).strip()
                        if word == '':
                            continue
                        #print(word)
                        term = word.lower()
                        dictionary_dic.setdefault(term,{})
                        if docID not in dictionary_dic[term]:
                            dictionary_dic[term][docID] = idx
                        #term = morph.parse(term)[0].normal_form
                        if(self.param_dict['IGNORE_SINGLE'] and len(term) == 1):
                            continue
                        
                        if term not in dictionary:
                            dictionary[term] = [docID]
                        else:
                            if(dictionary[term][-1] != docID):
                                dictionary[term].append(docID)
                file.close()
        dict_file = codecs.open(self.dictionary_file,'w',encoding='utf-8')
        post_file = open(self.postings_file,'wb')
        with open('data\sent_pos.pickle','wb') as stream:
            pickle.dump(dictionary_dic,stream)
        byte_offset = 0
        dict_file.write('Indexed from docIDs:')
        for docID in docID_list:
            dict_file.write(str(docID) + ',')
        dict_file.write('\n')  
        for term, postings in dictionary.items():
            block_size = len(postings)
            for docID in postings:
                post_file.write(struct.pack('H',docID))       
            dict_file.write(term + ' ' + str(block_size) + ' ' + str(byte_offset) + '\n')
            byte_offset += self.param_dict['BYTE_OFFSET'] * block_size
        dict_file.close()
        post_file.close()
        
    def position_index(self):
        docID_list = [idx for idx in range(39999)]
        dictionary = {}
        #rework
        for docID in docID_list:
            file_path = self.document_directory + str(docID)
            file_path+='.txt'
            if(os.path.isfile(file_path)):
                file = codecs.open(file_path,encoding = 'utf-8')
                document = file.read()
                tokens = document.split(' ')
                t = []
                for token in tokens:
                    t.extend(token.split('-'))
                tokens = t
                for idx in range(len(tokens)):
                    token = re.sub('\W+','',tokens[idx])
                    token = token.lower().strip()
                    if token == '':
                        continue
                    if docID == 7:
                        print(token)
                    dictionary.setdefault(token,{})
                    dictionary[token].setdefault(docID,[])
                    dictionary[token][docID].append(idx)   
                file.close()
        dict_file = codecs.open(self.dictionary_file,'w',encoding='utf-8')
        post_file = open(self.postings_file,'wb')
        
        byte_offset = 0
        dict_file.write('Indexed from docIDs:')
        for docID in docID_list:
            dict_file.write(str(docID) + ',')
        dict_file.write('\n')
        for term,postings in dictionary.items():
            block_size = len(postings)
            summer = 0
            for docID in sorted(postings.keys()):
                block_size_pos = len(postings[docID])
                summer += block_size_pos 
                post_file.write(struct.pack('H',docID))
                post_file.write(struct.pack('H',block_size_pos))
                for item in postings[docID]:
                    post_file.write(struct.pack('H',item))
            dict_file.write(term + ' ' + str(block_size) + ' ' + str(byte_offset) + '\n')
            byte_offset += 4 * block_size + 2 * summer 
        dict_file.close()
        post_file.close()
        
    def tf_idf_pos(self):
        docID_list = [idx for idx in range(39999)]
        dictionary = {}
        dict_len = {}
        #rework
        for docID in docID_list:
            file_path = self.document_directory + str(docID)
            file_path+='.txt'
            if(os.path.isfile(file_path)):
                file = codecs.open(file_path,encoding = 'utf-8')
                document = file.read()
                tokens = document.split(' ')
                t = []
                for token in tokens:
                    t.extend(token.split('-'))
                tokens = t
                for idx in range(len(tokens)):
                    token = re.sub('\W+','',tokens[idx])
                    if token == '':
                        continue
                    token = tokens[idx].lower()
                    #token = morph.parse(token)[0].normal_form
                    dictionary.setdefault(token,{})
                    dictionary[token].setdefault(docID,[])
                    dictionary[token][docID].append(idx)
                dict_len[docID] = len(tokens)
                file.close()
        dict_file = codecs.open(self.dictionary_file,'w',encoding='utf-8')
        post_file = open(self.postings_file,'wb')
        byte_offset = 0
        dict_file.write('Indexed from docIDs:')
        for docID in docID_list:
            dict_file.write(str(docID) + ',')
        dict_file.write('\n')
        for term,postings in dictionary.items():
            block_size = len(postings)
            idf = math.log10(40000/block_size)
            summer = 0
            for docID in sorted(postings.keys()):
                block_size_pos = len(postings[docID])
                if term == 'литва':
                    print(block_size_pos)
                tf = 1 + math.log10(block_size_pos)
                summer += block_size_pos 
                post_file.write(struct.pack('I',docID))
                #post_file.write(struct.pack('I',block_size_pos))
                post_file.write(struct.pack('d',tf*idf))
                #for item in postings[docID]:
                    #post_file.write(struct.pack('I',item))
            dict_file.write(term + ' ' + str(block_size) + ' ' + str(byte_offset) + '\n')
            byte_offset +=  4 * block_size  + 8 * block_size
        dict_file.close()
        post_file.close()

In [86]:
search = BooleanIndexer('text\wiki','dictionary.txt','postings.txt')

In [87]:
start = timeit.default_timer()
search = BooleanIndexer(r'data\text\wiki',r'data\dictionary.txt',r'data\postings.txt')
search.boolean_index()
end = timeit.default_timer()
print(end-start)

0
data\text\wiki0.txt
1
data\text\wiki1.txt
2
data\text\wiki2.txt
3
data\text\wiki3.txt
4
data\text\wiki4.txt
5
data\text\wiki5.txt
6
data\text\wiki6.txt
7
data\text\wiki7.txt
8
data\text\wiki8.txt
9
data\text\wiki9.txt
10
data\text\wiki10.txt
11
data\text\wiki11.txt
12
data\text\wiki12.txt
13
data\text\wiki13.txt
14
data\text\wiki14.txt
15
data\text\wiki15.txt
16
data\text\wiki16.txt
17
data\text\wiki17.txt
18
data\text\wiki18.txt
19
data\text\wiki19.txt
20
data\text\wiki20.txt
21
data\text\wiki21.txt
22
data\text\wiki22.txt
23
data\text\wiki23.txt
24
data\text\wiki24.txt
25
data\text\wiki25.txt
26
data\text\wiki26.txt
27
data\text\wiki27.txt
28
data\text\wiki28.txt
29
data\text\wiki29.txt
30
data\text\wiki30.txt
31
data\text\wiki31.txt
32
data\text\wiki32.txt
33
data\text\wiki33.txt
34
data\text\wiki34.txt
35
data\text\wiki35.txt
36
data\text\wiki36.txt
37
data\text\wiki37.txt
38
data\text\wiki38.txt
39
data\text\wiki39.txt
40
data\text\wiki40.txt
41
data\text\wiki41.txt
42
data\text

367
data\text\wiki367.txt
368
data\text\wiki368.txt
369
data\text\wiki369.txt
370
data\text\wiki370.txt
371
data\text\wiki371.txt
372
data\text\wiki372.txt
373
data\text\wiki373.txt
374
data\text\wiki374.txt
375
data\text\wiki375.txt
376
data\text\wiki376.txt
377
data\text\wiki377.txt
378
data\text\wiki378.txt
379
data\text\wiki379.txt
380
data\text\wiki380.txt
381
data\text\wiki381.txt
382
data\text\wiki382.txt
383
data\text\wiki383.txt
384
data\text\wiki384.txt
385
data\text\wiki385.txt
386
data\text\wiki386.txt
387
data\text\wiki387.txt
388
data\text\wiki388.txt
389
data\text\wiki389.txt
390
data\text\wiki390.txt
391
data\text\wiki391.txt
392
data\text\wiki392.txt
393
data\text\wiki393.txt
394
data\text\wiki394.txt
395
data\text\wiki395.txt
396
data\text\wiki396.txt
397
data\text\wiki397.txt
398
data\text\wiki398.txt
399
data\text\wiki399.txt
400
data\text\wiki400.txt
401
data\text\wiki401.txt
402
data\text\wiki402.txt
403
data\text\wiki403.txt
404
data\text\wiki404.txt
405
data\tex

705
data\text\wiki705.txt
706
data\text\wiki706.txt
707
data\text\wiki707.txt
708
data\text\wiki708.txt
709
data\text\wiki709.txt
710
data\text\wiki710.txt
711
data\text\wiki711.txt
712
data\text\wiki712.txt
713
data\text\wiki713.txt
714
data\text\wiki714.txt
715
data\text\wiki715.txt
716
data\text\wiki716.txt
717
data\text\wiki717.txt
718
data\text\wiki718.txt
719
data\text\wiki719.txt
720
data\text\wiki720.txt
721
data\text\wiki721.txt
722
data\text\wiki722.txt
723
data\text\wiki723.txt
724
data\text\wiki724.txt
725
data\text\wiki725.txt
726
data\text\wiki726.txt
727
data\text\wiki727.txt
728
data\text\wiki728.txt
729
data\text\wiki729.txt
730
data\text\wiki730.txt
731
data\text\wiki731.txt
732
data\text\wiki732.txt
733
data\text\wiki733.txt
734
data\text\wiki734.txt
735
data\text\wiki735.txt
736
data\text\wiki736.txt
737
data\text\wiki737.txt
738
data\text\wiki738.txt
739
data\text\wiki739.txt
740
data\text\wiki740.txt
741
data\text\wiki741.txt
742
data\text\wiki742.txt
743
data\tex

1053
data\text\wiki1053.txt
1054
data\text\wiki1054.txt
1055
data\text\wiki1055.txt
1056
data\text\wiki1056.txt
1057
data\text\wiki1057.txt
1058
data\text\wiki1058.txt
1059
data\text\wiki1059.txt
1060
data\text\wiki1060.txt
1061
data\text\wiki1061.txt
1062
data\text\wiki1062.txt
1063
data\text\wiki1063.txt
1064
data\text\wiki1064.txt
1065
data\text\wiki1065.txt
1066
data\text\wiki1066.txt
1067
data\text\wiki1067.txt
1068
data\text\wiki1068.txt
1069
data\text\wiki1069.txt
1070
data\text\wiki1070.txt
1071
data\text\wiki1071.txt
1072
data\text\wiki1072.txt
1073
data\text\wiki1073.txt
1074
data\text\wiki1074.txt
1075
data\text\wiki1075.txt
1076
data\text\wiki1076.txt
1077
data\text\wiki1077.txt
1078
data\text\wiki1078.txt
1079
data\text\wiki1079.txt
1080
data\text\wiki1080.txt
1081
data\text\wiki1081.txt
1082
data\text\wiki1082.txt
1083
data\text\wiki1083.txt
1084
data\text\wiki1084.txt
1085
data\text\wiki1085.txt
1086
data\text\wiki1086.txt
1087
data\text\wiki1087.txt
1088
data\text\wiki1

1413
data\text\wiki1413.txt
1414
data\text\wiki1414.txt
1415
data\text\wiki1415.txt
1416
data\text\wiki1416.txt
1417
data\text\wiki1417.txt
1418
data\text\wiki1418.txt
1419
data\text\wiki1419.txt
1420
data\text\wiki1420.txt
1421
data\text\wiki1421.txt
1422
data\text\wiki1422.txt
1423
data\text\wiki1423.txt
1424
data\text\wiki1424.txt
1425
data\text\wiki1425.txt
1426
data\text\wiki1426.txt
1427
data\text\wiki1427.txt
1428
data\text\wiki1428.txt
1429
data\text\wiki1429.txt
1430
data\text\wiki1430.txt
1431
data\text\wiki1431.txt
1432
data\text\wiki1432.txt
1433
data\text\wiki1433.txt
1434
data\text\wiki1434.txt
1435
data\text\wiki1435.txt
1436
data\text\wiki1436.txt
1437
data\text\wiki1437.txt
1438
data\text\wiki1438.txt
1439
data\text\wiki1439.txt
1440
data\text\wiki1440.txt
1441
data\text\wiki1441.txt
1442
data\text\wiki1442.txt
1443
data\text\wiki1443.txt
1444
data\text\wiki1444.txt
1445
data\text\wiki1445.txt
1446
data\text\wiki1446.txt
1447
data\text\wiki1447.txt
1448
data\text\wiki1

...

38504
data\text\wiki38504.txt
38505
data\text\wiki38505.txt
38506
data\text\wiki38506.txt
38507
data\text\wiki38507.txt
38508
data\text\wiki38508.txt
38509
data\text\wiki38509.txt
38510
data\text\wiki38510.txt
38511
data\text\wiki38511.txt
38512
data\text\wiki38512.txt
38513
data\text\wiki38513.txt
38514
data\text\wiki38514.txt
38515
data\text\wiki38515.txt
38516
data\text\wiki38516.txt
38517
data\text\wiki38517.txt
38518
data\text\wiki38518.txt
38519
data\text\wiki38519.txt
38520
data\text\wiki38520.txt
38521
data\text\wiki38521.txt
38522
data\text\wiki38522.txt
38523
data\text\wiki38523.txt
38524
data\text\wiki38524.txt
38525
data\text\wiki38525.txt
38526
data\text\wiki38526.txt
38527
data\text\wiki38527.txt
38528
data\text\wiki38528.txt
38529
data\text\wiki38529.txt
38530
data\text\wiki38530.txt
38531
data\text\wiki38531.txt
38532
data\text\wiki38532.txt
38533
data\text\wiki38533.txt
38534
data\text\wiki38534.txt
38535
data\text\wiki38535.txt
38536
data\text\wiki38536.txt
38537
data

38791
data\text\wiki38791.txt
38792
data\text\wiki38792.txt
38793
data\text\wiki38793.txt
38794
data\text\wiki38794.txt
38795
data\text\wiki38795.txt
38796
data\text\wiki38796.txt
38797
data\text\wiki38797.txt
38798
data\text\wiki38798.txt
38799
data\text\wiki38799.txt
38800
data\text\wiki38800.txt
38801
data\text\wiki38801.txt
38802
data\text\wiki38802.txt
38803
data\text\wiki38803.txt
38804
data\text\wiki38804.txt
38805
data\text\wiki38805.txt
38806
data\text\wiki38806.txt
38807
data\text\wiki38807.txt
38808
data\text\wiki38808.txt
38809
data\text\wiki38809.txt
38810
data\text\wiki38810.txt
38811
data\text\wiki38811.txt
38812
data\text\wiki38812.txt
38813
data\text\wiki38813.txt
38814
data\text\wiki38814.txt
38815
data\text\wiki38815.txt
38816
data\text\wiki38816.txt
38817
data\text\wiki38817.txt
38818
data\text\wiki38818.txt
38819
data\text\wiki38819.txt
38820
data\text\wiki38820.txt
38821
data\text\wiki38821.txt
38822
data\text\wiki38822.txt
38823
data\text\wiki38823.txt
38824
data

39133
data\text\wiki39133.txt
39134
data\text\wiki39134.txt
39135
data\text\wiki39135.txt
39136
data\text\wiki39136.txt
39137
data\text\wiki39137.txt
39138
data\text\wiki39138.txt
39139
data\text\wiki39139.txt
39140
data\text\wiki39140.txt
39141
data\text\wiki39141.txt
39142
data\text\wiki39142.txt
39143
data\text\wiki39143.txt
39144
data\text\wiki39144.txt
39145
data\text\wiki39145.txt
39146
data\text\wiki39146.txt
39147
data\text\wiki39147.txt
39148
data\text\wiki39148.txt
39149
data\text\wiki39149.txt
39150
data\text\wiki39150.txt
39151
data\text\wiki39151.txt
39152
data\text\wiki39152.txt
39153
data\text\wiki39153.txt
39154
data\text\wiki39154.txt
39155
data\text\wiki39155.txt
39156
data\text\wiki39156.txt
39157
data\text\wiki39157.txt
39158
data\text\wiki39158.txt
39159
data\text\wiki39159.txt
39160
data\text\wiki39160.txt
39161
data\text\wiki39161.txt
39162
data\text\wiki39162.txt
39163
data\text\wiki39163.txt
39164
data\text\wiki39164.txt
39165
data\text\wiki39165.txt
39166
data

39463
data\text\wiki39463.txt
39464
data\text\wiki39464.txt
39465
data\text\wiki39465.txt
39466
data\text\wiki39466.txt
39467
data\text\wiki39467.txt
39468
data\text\wiki39468.txt
39469
data\text\wiki39469.txt
39470
data\text\wiki39470.txt
39471
data\text\wiki39471.txt
39472
data\text\wiki39472.txt
39473
data\text\wiki39473.txt
39474
data\text\wiki39474.txt
39475
data\text\wiki39475.txt
39476
data\text\wiki39476.txt
39477
data\text\wiki39477.txt
39478
data\text\wiki39478.txt
39479
data\text\wiki39479.txt
39480
data\text\wiki39480.txt
39481
data\text\wiki39481.txt
39482
data\text\wiki39482.txt
39483
data\text\wiki39483.txt
39484
data\text\wiki39484.txt
39485
data\text\wiki39485.txt
39486
data\text\wiki39486.txt
39487
data\text\wiki39487.txt
39488
data\text\wiki39488.txt
39489
data\text\wiki39489.txt
39490
data\text\wiki39490.txt
39491
data\text\wiki39491.txt
39492
data\text\wiki39492.txt
39493
data\text\wiki39493.txt
39494
data\text\wiki39494.txt
39495
data\text\wiki39495.txt
39496
data

39818
data\text\wiki39818.txt
39819
data\text\wiki39819.txt
39820
data\text\wiki39820.txt
39821
data\text\wiki39821.txt
39822
data\text\wiki39822.txt
39823
data\text\wiki39823.txt
39824
data\text\wiki39824.txt
39825
data\text\wiki39825.txt
39826
data\text\wiki39826.txt
39827
data\text\wiki39827.txt
39828
data\text\wiki39828.txt
39829
data\text\wiki39829.txt
39830
data\text\wiki39830.txt
39831
data\text\wiki39831.txt
39832
data\text\wiki39832.txt
39833
data\text\wiki39833.txt
39834
data\text\wiki39834.txt
39835
data\text\wiki39835.txt
39836
data\text\wiki39836.txt
39837
data\text\wiki39837.txt
39838
data\text\wiki39838.txt
39839
data\text\wiki39839.txt
39840
data\text\wiki39840.txt
39841
data\text\wiki39841.txt
39842
data\text\wiki39842.txt
39843
data\text\wiki39843.txt
39844
data\text\wiki39844.txt
39845
data\text\wiki39845.txt
39846
data\text\wiki39846.txt
39847
data\text\wiki39847.txt
39848
data\text\wiki39848.txt
39849
data\text\wiki39849.txt
39850
data\text\wiki39850.txt
39851
data

In [88]:
start = timeit.default_timer()
search = BooleanIndexer(r'data\text\wiki',r'data\dictionary_pos.txt',r'data\postings_pos.txt')
search.position_index()
end = timeit.default_timer()
print(end-start)

эшка
смесь
двух
частей
mgo
и
одной
части
na2co3
реагент
хорошо
поглощающий
окислы
серы
и
хлора
например
для
определения
содержания
серы
в
угле
навеску
угля
сжигают
со
смесью
эшка
при
этом
образуются
растворимые
сульфаты
магния
и
натрия
их
растворяют
в
горячей
воде
и
затем
добавляют
хлорид
нитрат
бария
по
количеству
осадка
baso4
судят
о
количестве
серы
в
угле
вероятно
этот
способ
был
предложен
вещества
88.1299912239997


In [75]:
start = timeit.default_timer()
search = BooleanIndexer(r'data\text\wiki',r'data\dictionary_tf.txt',r'data\postings_tf.txt')
search.tf_idf_pos()
end = timeit.default_timer()
print(end-start)

1
1
1
1
1
1
2
1
1
4
1
1
1
1
1
3
4
1
1
1
1
1
3
1
1
2
1
1
1
1
5
1
2
1
1
1
2
1
1
1
6
1
5
1
1
1
1
4
1
1
2
1
1
1
3
3
1
1
5
1
1
2
1
2
10
10
1
3
1
3
1
158.74239867599863


In [2]:
class Search(object):
    def __init__(self,dictionary_file,postings_file,dictionary_file_tf, postings_file_tf):
        self.df = dictionary_file
        self.pf = postings_file
        self.df_tf = dictionary_file_tf
        self.pf_tf = postings_file_tf
    
    def load_dictionary(self):
        dict_stream = codecs.open(self.df,encoding = 'utf-8')
        dictionary = {}
        docIDs = []
        processed = False
        for entry in dict_stream.read().split('\n'):
            if(entry):
                if not processed:
                    #print(entry[20:-1])
                    docIDs = [int(docID) for docID in entry[20:-1].split(',')]
                    processed = True
                else:
                    token = entry.split(' ')
                    term = token[0]
                    df = int(token[1])
                    offset = int(token[2])
                    dictionary[term] = (df,offset)
        dict_stream.close()           
        return (dictionary,docIDs)
    
    def load_dictionary_pos(self):
        dict_stream = codecs.open(self.df,encoding = 'utf-8')
        dictionary = {}
        docIDs = []
        processed = False
        for entry in dict_stream.read().split('\n'):
            if(entry):
                if not processed:
                    #print(entry[20:-1])
                    docIDs = [int(docID) for docID in entry[20:-1].split(',')]
                    processed = True
                else:
                    token = entry.split(' ')
                    term = token[0]
                    df = int(token[1])
                    offset = int(token[2])
                    dictionary[term] = (df,offset)
        dict_stream.close()           
        return (dictionary,docIDs)
    
    def load_dictionary_tf(self):
        dict_stream = codecs.open(self.df_tf,encoding = 'utf-8')
        dictionary = {}
        docIDs = []
        processed = False
        for entry in dict_stream.read().split('\n'):
            if(entry):
                if not processed:
                    #print(entry[20:-1])
                    docIDs = [int(docID) for docID in entry[20:-1].split(',')]
                    processed = True
                else:
                    token = entry.split(' ')
                    term = token[0]
                    df = int(token[1])
                    offset = int(token[2])
                    dictionary[term] = (df,offset)
        dict_stream.close()           
        return (dictionary,docIDs)
    
    def load_postings_tf(self,term,length,offset):
        dict_tf_idf = {}
        dict_tf_idf[term] = {}
        post_file = open(self.pf_tf,'rb')
        post_file.seek(offset)
        for idx in range(length):
            a = post_file.read(4)
            docID = struct.unpack('I',a)[0]
            #print(a)
            c = post_file.read(8)
            tf_idf = struct.unpack('d',c)[0]
            #print('tf_idf',tf_idf)
            dict_tf_idf[term][docID] = tf_idf
        return dict_tf_idf
    
    def ranger(self,requests,answer):
        dictionary = self.load_dictionary_tf()[0]
        tf_idf_dicts = []
        for term in requests:
            tf_idf_dicts.append(self.load_postings_tf(term,dictionary[term][0],dictionary[term][1]))
        tf = []
        for idx in range(len(answer)):
            res = 0
            for jdx in range(len(tf_idf_dicts)):
                term_ =list(tf_idf_dicts[jdx].keys())[0]
                if answer[idx] not in tf_idf_dicts[jdx][term_]:
                    #print('Here')
                    continue
                else:
                    res += tf_idf_dicts[jdx][term_][answer[idx]]
            #print(res)
            tf.append((answer[idx],res))
        return sorted(list(set(tf)),key = lambda tup: tup[1],reverse = True)
                    
                
    
    def load_postings(self,length,offset):
        post_file = codecs.open(self.pf,'rb')
        post_file.seek(offset)
        postings = []
        for i in range(length):
            posting = post_file.read(param['BYTE_SIZE'])
            docID = struct.unpack('H',posting)[0]
            postings.append(docID)
        post_file.close()
        print(len(postings))
        return postings
    
    def load_postings_with_pos(self,length,offset):
        postings = {}
        post_file = open(self.pf,'rb')
        post_file.seek(offset)
        for idx in range(length):
            a = post_file.read(2)
            docID = struct.unpack('H',a)[0]
            #print(docID)
            b = post_file.read(2)
            lent = struct.unpack('H',b)[0]
            #print(b)
            postings[docID] = []
            for idx in range(lent):
                c = post_file.read(2)
                c = struct.unpack('H',c)[0]
                postings[docID].append(c)
        post_file.close()
        return postings
        
    #with skips
    def intersect(self,left_op,right_op):
        result = []
        idx = 0
        jdx = 0
        l_skip = int(math.sqrt(len(left_op)))
        r_skip = int(math.sqrt(len(right_op)))
        while (idx < len(left_op)) and (jdx < len(right_op)):
            if left_op[idx] == right_op[jdx]:
                result.append(left_op[idx])
                idx+=1
                jdx+=1
            elif left_op[idx] > right_op[jdx]:
                if (jdx + r_skip < len(right_op)) and (right_op[jdx + r_skip] <= left_op[idx]):
                    jdx+= r_skip
                else:
                    jdx+=1
            else:
                if (idx + l_skip < len(left_op)) and (left_op[idx + l_skip] <= right_op[jdx]):
                    idx += l_skip
                else:
                    idx+=1
        return result
    
    def position_intersect(self,left_op,right_op,intersected_keys,distance):
        answer = []
        dict_answer = {}
        #print(left_op)
        for docID in intersected_keys:
            l = []
            pp1 = sorted(left_op[docID])
            pp2 = sorted(right_op[docID])
            left_plen = len(pp1)
            right_plen = len(pp2)
            idx = 0
            jdx = 0
            while idx != left_plen:
                while jdx != right_plen:
                    if abs(pp1[idx] - pp2[jdx]) <= distance:
                        l.append(pp2[jdx])
                    elif pp2[jdx] > pp1[idx]:
                        break
                    jdx += 1
                while l != [] and abs(l[0] - pp1[idx]) > distance:
                    l.remove(l[0])
                for ps in l:
                    dict_answer.setdefault(docID,[])
                    answer.append([docID,pp1[idx],ps])
                    dict_answer[docID].append(pp1[idx])
                idx +=1
        return answer,dict_answer
    
    def full_enterance_citate(self,tokens):
        #print(tokens)
        answer = tokens[0]
        intersected_keys = set(answer.keys())
        #print(7 in intersected_keys)
        postings = []
        #print(len(tokens))
        for idx in range(1,len(tokens)):
            intersected_keys = intersected_keys.intersection(set(tokens[idx].keys()))
        intersected_keys = list(intersected_keys)
        #print('len of intersection',len(intersected_keys))
        k = 1
        for idx in range(1,len(tokens)):
            #print(idx)
            _,answer = self.position_intersect(answer,tokens[idx],intersected_keys,k)
            intersected_keys = answer.keys()
            k += 1
        #print(answer.keys())
        return sorted(list(answer.keys()))
            
    
                    
    def union(self,left_op,right_op):
        result = []
        idx = 0
        jdx = 0
        while (idx < len(left_op)) or (jdx < len(right_op)):
            if (idx < len(left_op)) and (jdx < len(right_op)):
                if left_op[idx] == right_op[jdx]:
                    result.append(left_op[idx])
                    idx+=1
                    jdx+=1
                    
                elif left_op[idx] < right_op[jdx]:
                    result.append(left_op[idx])
                    idx+=1
                else:
                    result.append(right_op[jdx])
                    jdx+=1
            elif idx >= len(left_op):
                result.append(right_op[jdx])
                jdx+=1
            else:
                result.append(left_op[idx])
                idx+=1
        return result
    
    def inverse(self,op,docIDs):
        if not op:
            return docIDs
        print(op)
        result = [item for item in docIDs if item not in op]
        return result
    
    def shunting_yard(self,infix_tokens):
        precedence = {}
        precedence['NOT'] = 3
        precedence['AND'] = 2
        precedence['OR'] = 1
        precedence['('] = 0
        precedence[')'] = 0
        output = []
        operator_stack = []
        for token in infix_tokens:
            if token == '(':
                operator_stack.append(token)
            elif token == ')':
                operator = operator_stack.pop()
                while operator != '(':
                    output.append(operator)
                    operator = operator_stack.pop()
            elif token in precedence:
                if operator_stack:
                    current_operator = operator_stack[-1]
                    while operator_stack and precedence[current_operator] > precedence[token]:
                        output.append(operator_stack.pop())
                        if operator_stack:
                            current_operator = operator_stack[-1]
                            
                operator_stack.append(token)
            else:
                output.append(token.lower())
                
        while operator_stack:
            output.append(operator_stack.pop())
        return output
            
    
    def process_query(self,query,docIDs,dictionary_pos):
        query = query.replace('(', '( ')
        query = query.replace(')',' )')
        query = query.split(' ')
        result_stack = []
        postfix_queue = collections.deque(self.shunting_yard(query))
        queue = postfix_queue.copy()
        queue = list(set(queue))
        try:
            queue.remove('AND')
        except:
            pass
        try:
            queue.remove('OR')
        except:
            pass
        try:
            queue.remove('NOT')
        except:
            pass
        try:
            queue.remove('<')
        except:
            pass
        try:
            queue.remove('>')
        except:
            pass 
                
        #print(queue)
        #print(postfix_queue)
        while postfix_queue:
            token = postfix_queue.popleft()
            #print(token)
            result = []
            if(token != 'AND' and token != 'OR' and token != 'NOT' and token != '<'):
                if token in dictionary_pos:
                    result = sorted(list(self.load_postings_with_pos(dictionary_pos[token.strip()][0],dictionary_pos[token.strip()][1]).keys()))
                    another_variations = []
                    idx = 0
                    for key in dictionary_pos.keys():
                        if morph.parse(key)[0].normal_form == morph.parse(token)[0].normal_form:
                            #print('morphology')
                            result.extend(sorted(list(self.load_postings_with_pos(dictionary_pos[key][0],dictionary_pos[key][1]).keys())))
                            queue.append(key)
                            idx += 1 
                            if idx == 3:
                                break
                    result = sorted(result)
            elif token == 'AND':
                right_operand = result_stack.pop()
                left_operand = result_stack.pop()
                result = self.intersect(left_operand,right_operand)
                    
            elif token == 'OR':
                right_operand = result_stack.pop()
                left_operand = result_stack.pop()
                result = self.union(left_operand,right_operand)
                    
            elif token == 'NOT':
                right_operand = result_stack.pop()
                #print(right_operand)
                result = self.inverse(right_operand,docIDs)
                
            elif token == '<':
                citate = []
                operand = postfix_queue.popleft()
                while '>' not in operand:
                    #print('Citate')
                    operand = operand.strip()
                    citate.append(self.load_postings_with_pos(dictionary_pos[operand][0],dictionary_pos[operand][1]))
                    operand = postfix_queue.popleft()
                if operand == '>':
                    c = 1
                else:
                    c = int(operand[-1])
                if c == 1:
                    result = self.full_enterance_citate(citate)
                #print('res',result)
                
            result_stack.append(result)
                
        if len(result_stack) != 1:
            print(len(result_stack))
            print ("ERROR: results_stack. Please check valid query")
        answer = self.ranger(queue,result_stack[-1])
        #return result_stack.pop()
        return answer,queue
    
    def build_text_markup(self,query,docID,l = 2):
        #print(docID)
        string = ''
        idx = 0
        prev = -1
        with open(r'headers.pickle','rb') as stream:
            info = pickle.load(stream)
        with open(r'data\sent_pos.pickle','rb') as stream:
            readers = pickle.load(stream)
        with open(r'data\text\wiki{0}.txt'.format(docID),'r',encoding = 'utf-8') as stream:
            text = stream.read().split('.')
        for word in query:
            if docID in readers[word]:
                if idx == l:
                    break;
                if prev == readers[word][docID]:
                    continue
                string += text[readers[word][docID]]
                prev = readers[word][docID]
                idx+=1
        header = info[docID]
        if len(string) > 300:
            string = string[:300]
            string+='.....'
        output_text = HTML('<p>{0}</p>'.format(string))
        header_text = HTML('<h3>{0}</h3>'.format(header))
        return header_text, output_text
                
                
            
    def search_bool(self,query):
        docIDs = [range(39999)]
        loaded_dict_pos = self.load_dictionary_pos()
        dictionary_pos = loaded_dict_pos[0]
        result,queue = self.process_query(query,docIDs,dictionary_pos)
        if len(result) == 1:
            ss = 'статья'
        elif len(result) >= 2 and len(result) <=4:
            ss = 'статьи'
        else:
            ss = 'статей'
        display(HTML('По запросу {0} было найдено {1} {2}'.format(query,len(result),ss)))
        #print(result[:10])
        for idx in range(len(result[:10])):
            docID = result[idx][0]
            h,t = self.build_text_markup(queue,docID)
            display(h)
            display(t)
        

In [3]:
a = Search(r"data\dictionary_pos.txt",r"data\postings_pos.txt",r"data\dictionary_tf.txt",r"data\postings_tf.txt")

In [5]:
a.search_bool('< смесь двух частей >')

In [41]:
a.search_bool('крымский AND мост')

In [42]:
a.search_bool('крымский OR мост')

In [4]:
a.search_bool("чемпионат AND мира")